<a href="https://colab.research.google.com/github/bbcx-investments/notebooks/blob/main/risk/continuous_compounding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import DataReader as pdr


# annual market return data
df = pdr('F-F_Research_Data_Factors','famafrench', start=1900)[1]/100

mkt = df["Mkt-RF"] + df.RF
mkt = pd.DataFrame(mkt).reset_index()
mkt.columns = ['date', 'ret']
mkt.date = mkt.date.astype(str)

mkt.head(10)

,date,ret
0,1927,0.3259
1,1928,0.3895
2,1929,-0.1479
3,1930,-0.2882
4,1931,-0.4404
5,1932,-0.0843
6,1933,0.5735
7,1934,0.0318
8,1935,0.4513
9,1936,0.3225


In [ ]:
# individual stock annual return data
tick = 'CVX'        # example stock
close = pdr(tick, "yahoo", start=1970)["Adj Close"]
annual = close.resample('Y').last().pct_change().dropna().reset_index()
annual.columns = ['date', 'ret']
annual.date = annual.date.dt.to_period('Y').astype(str)

annual.head(10)

,date,ret
0,1971,0.106523
1,1972,0.453690
2,1973,-0.090196
3,1974,-0.316028
4,1975,0.417095
5,1976,0.481888
6,1977,-0.051829
7,1978,0.205788
8,1979,0.202667
9,1980,0.764967


In [ ]:
# Use market return data as examples to compute continuously compounded returns

mkt['log'] = np.log(1+mkt.ret)

mkt.head(10)

,date,ret,log
0,1927,0.3259,0.282091
1,1928,0.3895,0.328944
2,1929,-0.1479,-0.160051
3,1930,-0.2882,-0.339958
4,1931,-0.4404,-0.580533
5,1932,-0.0843,-0.088066
6,1933,0.5735,0.453302
7,1934,0.0318,0.031305
8,1935,0.4513,0.372460
9,1936,0.3225,0.279524


In [ ]:
# describe returns and continuous compounding returns data
tbl = mkt[['ret', 'log']].describe().iloc[1:]

tbl

,ret,log
mean,0.121726,0.097512
std,0.198931,0.193448
min,-0.440400,-0.580533
25%,0.000700,0.000700
50%,0.154000,0.143234
75%,0.269650,0.238741
max,0.573500,0.453302
